In [1]:
# Standard data analysis libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

# Web scraping libraries
from urllib.request import urlopen, Request
from urllib.error import HTTPError, URLError
import requests
import bs4

from bs4 import BeautifulSoup
import re
import os
from os.path import basename

# Some styling
%matplotlib inline
plt.style.use('fivethirtyeight')
mpl.rcParams['figure.figsize'] = (14, 8)
plt.rc('axes', titlesize=22) 
plt.rc('figure', titlesize=22)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('axes', labelsize=14)


In [2]:
URL_ALL_WATCHES = r"https://www.watchfinder.co.uk/all-watches?orderby=BestMatch&pageno="
URL_BASE = r"https://www.watchfinder.co.uk"

In [3]:
keys = ['brand',
        'series',
        'model',
        'price',
        'Year',
        'Box',
        'Papers',
        'Manufacturer warranty',
        'Watchfinder warranty',
        'Availability',
        'Delivery',
        'Returns policy',
        'Product code',
        'Movement',
        'Case size',
        'Case material',
        'Bracelet material',
        'Dial type',
        'Water resistance',
        'Location',
        'img_name']

In [6]:
def g(i):
    headers = {'User-Agent': 'Mozilla/5.0'}
    req = Request(URL_ALL_WATCHES + i, headers=headers)
    html = urlopen(req)
    bs = BeautifulSoup(html.read(), 'html.parser')
    
    a_tags = bs.find_all('a', {'class': 'prods_name redirect'})
    
    urls = []
    
    for i in range(len(a_tags)):
        urls.append(a_tags[i].attrs['href'])
        
    for i in range(len(urls)):
        urls[i] = urls[i].replace(' ', '%20')
    
    return urls

In [7]:
%%time

urls = []
for i in range(1, 115):
    urls = urls + g(str(i))

Wall time: 4min 15s


In [38]:
for i in range(len(urls)):
    if urls[i].find('é') > 1:
        urls[i] = urls[i].replace('é', '\\xe9')

In [9]:
def f(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    req = Request(url, headers=headers)
    html = urlopen(req)
    bs = BeautifulSoup(html.read(), 'html.parser')
    
    table_data = bs.find_all('tr')
    
    watches_dict = {}

    for key in keys:
        watches_dict[key] = None

    for i in range(len(table_data)):

        if len(list(table_data[i])) == 5:

            if list(table_data[i])[1].find('div'):
                key = list(list(table_data[i])[1].find('div'))[0]            
                value = list(list(table_data[i])[3])[0]

                watches_dict[key] = value

            else:
                key = list(list(table_data[i])[1])[0]            
                value = list(list(table_data[i])[3])[0]

                watches_dict[key] = value
                
    brand = list(bs.find('span', {'class': 'prod_brand ellipsis'}))[0]
    series = list(bs.find('span', {'class': 'prod_series ellipsis'}))[0]
    model = list(bs.find('span', {'class': 'prod_model ellipsis'}))[0]
    price = bs.find('meta', {'itemprop': 'price'}).attrs['content']
    
    watches_dict['brand'] = brand
    watches_dict['series'] = series
    watches_dict['model'] = model
    watches_dict['price'] = price
    
    image = bs.find('meta', {'itemprop': 'image'})
    img_url = image.attrs['content']
    img_name = basename(img_url)
    watches_dict['img_name'] = img_name
    save_path = os.path.join('./images/', img_name)
    with open(save_path, 'wb') as handle:
            response = requests.get(img_url, stream=True)

            if not response.ok:
                print(response)

            for block in response.iter_content(1024):
                if not block:
                    break

                handle.write(block)
                
    return watches_dict

In [39]:
%%time

dicts = []

for url in urls:
    dicts.append(f(URL_BASE + url))

KeyError: 'content'

In [40]:
big_dict = {}

for k in dicts[0]:
    big_dict[k] = [d[k] for d in dicts]

In [41]:
df = pd.DataFrame.from_dict(big_dict)

In [42]:
df.to_csv('wf_df_raw.csv')

In [ ]:
# df = pd.read_csv('wf_df_raw.csv')

In [24]:
for url in urls:
    if url.find('xe9') == 1:
        print(url)

In [37]:
for i in range(len(urls)):
    if urls[i].find('é') > 1:
        urls[i] = urls[i].replace('é', '\\xe9')

/Franck%20Muller/Cintrée%20Curvex/9880%20SC%20DT%20REL/44518/item/154152?searchId=875eb3f2-9b0d-43c3-8acc-3f1c3cf1c6d3&rank=13
/IWC/Pilots%20Chrono%20Antoine%20de%20St%20Exupéry/IW371711/36400/item/172079?searchId=290c2429-3117-40f7-a2c9-6c042bccc8eb&rank=9


In [36]:
urls[171].find('é')

-1

In [34]:
urls[172].replace('é', '\\xe9')

'/Franck%20Muller/Cintr\\xe9e%20Curvex/9880%20SC%20DT%20REL/44518/item/154152?searchId=875eb3f2-9b0d-43c3-8acc-3f1c3cf1c6d3&rank=13'